In [1]:
import  tensorflow as  tf 
import tensorflow.keras   as keras 
from keras.datasets import mnist
import numpy as np 

# use Keras to import pre-shuffled MNIST database
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("The MNIST database has a training set of %d examples." % len(X_train))
print("The MNIST database has a test set of %d examples." % len(X_test))


Using TensorFlow backend.


The MNIST database has a training set of 60000 examples.
The MNIST database has a test set of 10000 examples.


In [2]:
display ( X_train.shape ,X_test.shape , X_test.shape , y_test.shape)

(60000, 28, 28)

(10000, 28, 28)

(10000, 28, 28)

(10000,)

In [ ]:
X_train[0]

In [ ]:
import  matplotlib.pyplot as plt 
import matplotlib as mpl 


#### 灰度图展示一张

In [ ]:
plt.imshow( X_train[0]  ,cmap ="gray")

#### 灰度图展示多张

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

# plot first six training images
fig = plt.figure(figsize=(20,20))
for i in range(6):
    ax = fig.add_subplot(1, 6, i+1, xticks=[], yticks=[])
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(str(y_train[i]))

In [ ]:
X_train = X_train.astype(np.float) /255 

In [ ]:
X_test = X_test.astype(np.float) /255 

#### Integer-valued labels 

In [ ]:
y_train[0]

In [ ]:
y_hot_train  = tf.one_hot(y_train ,depth  =10 ) 
y_hot_test  = tf.one_hot(y_test ,depth  =10 ) 

#### One-hot labels:

In [ ]:
y_hot_train[0].numpy()

#### 图片的本质就是二维数据 ，就是矩阵

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(X_train[0], ax)

In [3]:
from sklearn.preprocessing import StandardScaler


x_valid, x_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    X_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

#### 建立模型

In [4]:
from tensorflow.keras import layers 
from tensorflow.keras import Sequential  
from tensorflow.keras.layers  import Dense ,Flatten ,Dropout  

model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))


In [5]:
# summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
__________________________________________________

#### Compile the Model

In [12]:
# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

###  Calculate the Classification Accuracy on the Test Set (Before Training)

In [13]:
# evaluate test accuracy
score = model.evaluate(X_test, y_test)  
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

10000/10000 [==============================] - 1s 66us/sample - loss: 105.0645 - accuracy: 0.0955
Test accuracy: 9.5500%


In [ ]:
X_test.shape 

#### fit the model 

In [ ]:
import os 
logdir = os.path.join("callbacks" )

if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"mnist_model.h5")


# callbacks 训练过程中
callbacks = [
    keras.callbacks.TensorBoard(logdir),  # 控制台 ，Ui 
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),  #只是保存最好的模型， 默认保存最后的模型
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),  #提前结束
]

epochs = 10 
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)
